In [13]:
import torch
import numpy as np
from sbi.inference import prepare_for_sbi, SNLE, SNPE
from sbi.simulators.linear_gaussian import diagonal_linear_gaussian
import sbi.utils as sbi_utils

### Load theta and x

In [2]:
data = np.load('../results/cleaned_up_data_onlyNumSpikes.npz', allow_pickle=True)

x = torch.as_tensor(data['data'], dtype=torch.float32)
theta = torch.as_tensor(data['params'], dtype=torch.float32)

In [3]:
x.shape

torch.Size([1994076, 5])

In [4]:
import sys
sys.path.append('../')

In [5]:
from parameter_setup import load_ground_truth_params, load_prior_min, load_prior_max

### Load data to standardize - needed to standardize x_o

In [6]:
#standardize_vals = np.load('../results/standardize_vals.npz')
#x_mean = standardize_vals['data_mean']
#x_std = standardize_vals['data_std']
#
#theta_mean = standardize_vals['theta_mean']
#theta_std = standardize_vals['theta_std']

x_mean = x.mean(dim=0)
x_std = x.std(dim=0)

theta_mean = theta.mean(dim=0)
theta_std = theta.std(dim=0)

In [7]:
x = (x-x_mean) / x_std
theta = (theta-theta_mean) / theta_std

### Load x_o (which I got from running the simulator with Arco's ground truth params)

In [8]:
import numpy as np
x_o = torch.as_tensor(np.load('../results/observation/x_o_new_ss.npz')['x_o'], dtype=torch.float32)
x_o = x_o[::7]
x_o = (x_o - x_mean) / x_std
x_o = x_o.unsqueeze(0)

### Create dummy simulator and dummy prior

In [9]:
def dummy_simulator(theta):
    return torch.ones(1,5)

dummy_prior = sbi_utils.BoxUniform(torch.as_tensor(load_prior_min()), torch.as_tensor(load_prior_max()))
_bound = torch.sqrt(torch.as_tensor(3.))
dummy_prior_norm = sbi_utils.BoxUniform(-_bound*torch.ones(35), _bound*torch.ones(35))

In [10]:
from sbi.utils.get_nn_models import likelihood_nn, posterior_nn

In [11]:
simulator, prior, x_shape = prepare_for_sbi(dummy_simulator, dummy_prior_norm)

nsf = likelihood_nn(
    model='nsf',
    theta_shape=prior.sample().shape,
    x_o_shape=x_shape,
    hidden_features=100,
    flow_num_transforms=5,
)

nsf_posterior = posterior_nn(
    model='nsf',
    prior=prior,
    x_o_shape=x_shape,
    hidden_features=100,
    flow_num_transforms=5,
)

/home/michael/anaconda3/envs/sbi/lib/python3.8/site-packages/nflows/transforms/standard.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "_shift", torch.tensor(shift if (shift is not None) else 0.0)
/home/michael/anaconda3/envs/sbi/lib/python3.8/site-packages/nflows/transforms/standard.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "_scale", torch.tensor(scale if (scale is not None) else 1.0)


In [42]:
inference = SNPE(
    simulator, 
    prior,
    x_shape,
    external_data=(theta, x), # todo: remove the [:1000] to train on all datasamples
    mcmc_method='slice', 
    density_estimator=nsf_posterior
)

In [43]:
posterior = inference(
    num_rounds=1,
    num_simulations_per_round=0,
    batch_size=100, # default is 50
    stop_after_epochs=5, # default is 20
)

Neural network successfully converged after 38 epochs.


In [44]:
import pickle
with open('../results/posteriors/200623_PosteriorSNPE_fitOnlyNumSpikes.pickle', 'wb') as handle:
    pickle.dump(posterior, handle)

### Run VI

In [45]:
from sbi.utils.get_nn_models import get_vi_net
from sbi.utils.vi import train_vi, train_mle
import torch
import sbi.utils as utils

In [46]:
import sys
sys.path.append('../../../bFlows')
from bflows.utils.neural_net.get_bounded_flows import get_bflow

SyntaxError: invalid syntax (get_bounded_flows.py, line 52)

# Temper the posterior

In [47]:
import numpy as np
x_o = torch.as_tensor(np.load('../results/observation/x_o_new_ss.npz')['x_o'], dtype=torch.float32)
x_o = x_o[::7]
x_o[0] = 2.

In [48]:
x_o = (x_o - x_mean) / x_std
x_o = x_o.unsqueeze(0)

In [49]:
x_o

tensor([[ 2.6027,  1.3153,  1.3030, -0.4727,  0.2522]])

# Train the VI

In [50]:
posterior = posterior.set_default_x(x_o)

### Analyse posterior

In [ ]:
import pickle
with open('../results/posteriors/200623_PosteriorSNPE_fitOnlyNumSpikes.pickle', 'rb') as handle:
    posterior = pickle.load(handle)

In [51]:
s = []
num_iter = 10
num_samples_per_iter = 1000

for k in range(num_iter):
    print("new iter,", k)
    samples = posterior.sample((num_samples_per_iter,))

    # denormalize the samples
    samples = samples * theta_std + theta_mean

    # convert to list for pickling
    samples_list = samples.numpy().tolist()
    
    s.append(samples_list)
s = np.asarray(s)
s = s.reshape(num_iter*num_samples_per_iter, -1)

new iter, 0
new iter, 1
new iter, 2
new iter, 3
new iter, 4
new iter, 5
new iter, 6
new iter, 7
new iter, 8
new iter, 9


In [52]:
s_list = s.tolist()

In [53]:
print(s_list[0])

[184.87274169921875, 0.032795608043670654, 866.86669921875, 0.024980919435620308, 214.65951538085938, 0.04490228742361069, 0.002000484149903059, 0.00020592767396010458, 0.00034289556788280606, 0.10562007874250412, 0.0034870579838752747, 0.0062654884532094, 0.00015373548376373947, 0.02225324511528015, 0.2154516577720642, 0.018768899142742157, 0.07078845053911209, 0.01912408135831356, 3.543728828430176, 3.351080894470215, 0.0015869250055402517, 0.008990748785436153, 0.00985052902251482, 0.021352890878915787, 0.08439171314239502, 0.02319522388279438, 0.9422140121459961, -1.3956375122070312, 1.5813851356506348, 1.268161654472351, 4.472703949431889e-05, 3.330173422000371e-05, 3.205982284271158e-05, 2.2045933292247355e-05, 1.3342845439910889]


In [54]:
with open('../results/samples/200622_PosteriorSNPE_onlyNumSpikes_samples.pickle', 'wb') as handle:
    pickle.dump(s_list, handle, protocol=2)

In [ ]:
gt_params = [137.862136034238,
 0.0005793520824526776,
 199.1298048149789,
 0.0006108049075983062,
 152.1647419393015,
 0.00742430653684668,
 0.0010965218089651857,
 0.0008186770602065786,
 0.00011435310571497434,
 0.0022763084379226854,
 0.0036986082079423594,
 0.00013883334761566004,
 3.2474860530531394e-06,
 0.005426837416265438,
 0.10568666421909532,
 0.04812676692103998,
 0.094826660872338,
 0.013854989311151315,
 3.9010342040060975,
 3.8851157748263354,
 0.009964343408409574,
 0.006000497448875096,
 0.0012602755616811401,
 0.01392240648099882,
 0.06283710421562513,
 6.68382138396179e-05,
 0.08311048073340864,
 -2.9836949894223825,
 1.9642986130169147,
 1.2999358521956366,
 4.4931548434199036e-05,
 2.062212836678345e-05,
 4.22059843297412e-05,
 2.2409802171891654e-05,
 1.7109080877160283]

In [ ]:
gt_params_norm = (torch.as_tensor(gt_params) - theta_mean) / theta_std

In [ ]:
samples = posterior.sample((10000,)).detach()

In [ ]:
probs = posterior.log_prob(samples).detach().numpy()
prob_of_gt = posterior.log_prob(gt_params_norm.unsqueeze(0)).detach().numpy()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1, figsize=(5,5))
_ = plt.hist(probs, bins=100)
ax.axvline(prob_of_gt, color='r')

In [33]:
dummy_prior_norm.log_prob(gt_params_norm.unsqueeze(0))

tensor([-43.4859])

### Posterior predictives
Has to be done in a different virtual env with python 2.